In [1]:
import numpy as np
import pandas as pd 

data = pd.read_csv("train.csv")
target = pd.read_csv("test.csv")

In [2]:
data.head()

,Unnamed: 0,FileName,XMin,XMax,YMin,YMax,ClassName
0,0,026319ec416e1d12.jpg,0.000,0.998333,0.000000,0.997778,Bed
1,1,117e0616bdbe2405.jpg,0.000,0.966250,0.731463,0.997996,Table
2,2,17af90ae78244f01.jpg,0.010,0.068750,0.484053,0.508443,Chair
3,3,17af90ae78244f01.jpg,0.015,0.078125,0.466229,0.522514,Chair
4,4,17af90ae78244f01.jpg,0.015,0.081875,0.499062,0.560976,Chair


In [3]:
target.head()

,Unnamed: 0,FileName,XMin,XMax,YMin,YMax,ClassName
0,0,180eaae399121a8c.jpg,0.218750,0.338125,0.510000,0.675833,Chair
1,1,180eaae399121a8c.jpg,0.757500,0.834375,0.594167,0.757500,Chair
2,2,180eaae399121a8c.jpg,0.795000,0.856875,0.513333,0.653333,Chair
3,3,180eaae399121a8c.jpg,0.311875,0.447500,0.227500,0.406667,Bed
4,4,180eaae399121a8c.jpg,0.636250,0.817500,0.230833,0.390000,Bed


In [4]:
data.drop(data.iloc[:,:1],axis=1,inplace=True)

In [5]:
data.head()

,FileName,XMin,XMax,YMin,YMax,ClassName
0,026319ec416e1d12.jpg,0.000,0.998333,0.000000,0.997778,Bed
1,117e0616bdbe2405.jpg,0.000,0.966250,0.731463,0.997996,Table
2,17af90ae78244f01.jpg,0.010,0.068750,0.484053,0.508443,Chair
3,17af90ae78244f01.jpg,0.015,0.078125,0.466229,0.522514,Chair
4,17af90ae78244f01.jpg,0.015,0.081875,0.499062,0.560976,Chair


In [6]:
target.drop(target.iloc[:,:1],axis=1,inplace=True)
target.head()

,FileName,XMin,XMax,YMin,YMax,ClassName
0,180eaae399121a8c.jpg,0.218750,0.338125,0.510000,0.675833,Chair
1,180eaae399121a8c.jpg,0.757500,0.834375,0.594167,0.757500,Chair
2,180eaae399121a8c.jpg,0.795000,0.856875,0.513333,0.653333,Chair
3,180eaae399121a8c.jpg,0.311875,0.447500,0.227500,0.406667,Bed
4,180eaae399121a8c.jpg,0.636250,0.817500,0.230833,0.390000,Bed


# REGION PROPOSAL NETWORK

In [7]:
def partial_vgg(input_tensor=None): 

    input_shape=(None,None,3) # we dont know the input shape right!
    i = Input(shape=input_shape) 
    
    # Block 1
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='block1_conv1')(i)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2,2), strides=(2,2), name='block1_pool')(x)
    print(x)
    
    # Block 2
    x = Conv2D(128, (3,3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2,2), strides=(2,2), name='block2_pool')(x)
    
    # Block 3
    x = Conv2D(256, (3,3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3,3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2,2), strides=(2,2), name='block3_pool')(x) 

    # Block 4
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2,2), strides=(2,2), name='block4_pool')(x) 
    
    # Block 5
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv3')(x)
#   x = MaxPooling2D((2,2), strides=(2,2), name='block5_pool')(x) 
    return x
# we are not using fully connected layer as we need only feature maps as output from this network


In [8]:
# RPN layer
def rpn_layer(base_layers, num_anchors):
    # cnn used for creating feature maps: vgg,num_anchors: 9
    x = Conv2D(512, (3,3), padding='same', activation='relu')(base_layers)
    
    # classification layer: num_anchors*2(9), channels for 0,1 sigmoid activation ouput
    x_class = Conv2D(num_anchors, (1,1), activation='sigmoid')(x)
    
    # reression layer: num_anchors*4 (36)channels for computing the regression of boxes
    x_regr = Conv2D(num_anchors * 4, (1,1), activation='linear')(x)
    
    return [x_class, x_regr, base_layers] # classification of object(0 or 1),compute bounding boxes,base layers vgg

In [9]:
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import math
import cv2
import copy
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import os

from sklearn.metrics import average_precision_score

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.objectives import categorical_crossentropy

from keras.models import Model
from keras.utils import generic_utils
from keras.engine import Layer, InputSpec
from keras import initializers, regularizers


In [11]:
class RoiPoolingConv(Layer):
    
    def _init_(self, pool_size, num_rois, **Kwargs):
        
        self.dim_ordering = k.image_dim_ordering()
        self.pool_size = pool_size
        self.num_rois = num_rois
        
        super(RoiPoolingConv, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.nb_channels = input_shape[0][3]
        
    def compute_output_shape(self, input_shape):
        return None, self.num_rois, self.pool_size, self.pool_size, self.nb_channels
    
    def call(self, x, mask=None):
        assert(len(x)==2)
        # x[0] is image with shape (rows, cols, channels)
        img = x[0]

        # x[1] is roi with shape (num_rois,4) with ordering (x,y,w,h)
        rois = x[1]
        
        input_shape = K.shape(img)

        outputs = []

        for roi_idx in range(self.num_rois):

            x = rois[0, roi_idx, 0]
            y = rois[0, roi_idx, 1]
            w = rois[0, roi_idx, 2]
            h = rois[0, roi_idx, 3]

            x = K.cast(x, 'int32')
            y = K.cast(y, 'int32')
            w = K.cast(w, 'int32')
            h = K.cast(h, 'int32')

            # Resized roi of the image to pooling size (7x7)
            rs = tf.image.resize_images(img[:, y:y+h, x:x+w, :], (self.pool_size, self.pool_size))
            outputs.append(rs)
                

        final_output = K.concatenate(outputs, axis=0)

        # Reshape to (1, num_rois, pool_size, pool_size, nb_channels) : (1, 4, 7, 7, 3)
        final_output = K.reshape(final_output, (1, self.num_rois, self.pool_size, self.pool_size, self.nb_channels))

        # permute_dimensions is similar to transpose
        final_output = K.permute_dimensions(final_output, (0, 1, 2, 3, 4))

        return final_output
    
    
    def get_config(self):
        config = {'pool_size': self.pool_size,
                                    'num_rois': self.num_rois}
        base_config = super(RoiPoolingConv, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [12]:
#Classifier layer

def classifier_layer(base_layers, input_rois, num_rois, nb_classes = 4):

    # base_layers: vgg
    #input_rois: `(1,num_rois,4)` list of rois, with ordering (x,y,w,h)
    #num_rois: number of rois to be processed in one time (4 in here)
    
    input_shape = (num_rois,7,7,512)

    pooling_regions = 7

    # out_roi_pool.shape = (1, num_rois, channels, pool_size, pool_size)
    # num_rois (4) 7x7 roi pooling
    out_roi_pool = RoiPoolingConv(pooling_regions, num_rois)([base_layers, input_rois])

    # Flatten the convlutional layer and connected to 2 FC and 2 dropout
    out = Flatten(name='flatten')(out_roi_pool) #expanded into a vector with 25,088 (7×7×512) channels.
    out = Dense(4096, activation='relu', name='fc1')(out)
    out = Dropout(0.5)(out)
    out = Dense(4096, activation='relu', name='fc2')(out)
    out = Dropout(0.5)(out)

    # two output layer- classifier and regressor
    # for classify the class name of the object
    out_class = Dense(nb_classes, activation='softmax', kernel_initializer='zero')(out)
    
    #for bboxes coordinates regression
    out_regr = Dense(4 * (nb_classes-1), activation='linear', kernel_initializer='zero')(out)

    return [out_class, out_regr]
